# Support vector machines soft margin

 - Informações sobre o conjunto de dados
    1. variance of Wavelet Transformed image (continuous)
    2. skewness of Wavelet Transformed image (continuous)
    3. curtosis of Wavelet Transformed image (continuous)
    4. entropy of image (continuous)
    5. class (integer)




In [1]:
import sys 

sys.path.append("/home/davi/PycharmProjects/Reconhecimento-de-Padroes")

from sklearn.model_selection import KFold
from Strings import string
from sklearn.model_selection import train_test_split
from numpy import zeros, identity, array, concatenate, \
    ones, ravel, dot, where
from src.Utils.utils import get_accuracy, get_data
import cvxopt
import cvxopt.solvers

In [2]:
def get_p(number_lines, number_columns):
     
    
    # tmp1 = concatenate((ones((number_columns - 1, 1)), zeros((number_columns - 1, 1))), axis=1)
    # tmp2 = zeros((1, number_columns - 1))
    # tmp3 = concatenate((tmp1, tmp2))
    # tmp4 = zeros((number_columns, 1))
    # tmp5 = concatenate((tmp3, tmp4), axis=1)
    # tmp_aux = array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])
    tmp_aux = identity(number_lines)
    tmp_aux[:, number_columns] = 0
    P = cvxopt.matrix(tmp_aux, tc='d')
    
    return P

In [3]:
def get_q(number_columns):
    tmp1 = zeros((number_columns, 1))
    Q = cvxopt.matrix(tmp1)
    
    return Q

In [4]:
def get_g(x, y, number_lines):
    tmp2 = array(y, ndmin=2).T * concatenate((x, ones((number_lines, 1))), axis=1)
    G = cvxopt.matrix(tmp2 * -1.)

    return G

In [5]:
def get_h(number_lines):
    tmp2 = ones((number_lines, 1))
    H = cvxopt.matrix(tmp2 * -1.)

    return H

In [6]:
def quadratic_solver(x_train, y_train):
    
    number_lines, number_columns = x_train.shape
    
    P = get_p(number_lines, number_columns)
    Q = get_q(number_columns)
    G = get_g(x_train, y_train, number_lines)
    H = get_h(number_lines)

    try:
        solution = cvxopt.solvers.qp(P, Q, G, H)
    except ValueError as error:
        return None

    best_weights = ravel(solution['x'])

    return best_weights


In [7]:
def train(x_train, y_train):
    solution = quadratic_solver(x_train, y_train)
    return solution

In [8]:
def get_foward(weights, x_test):
    H_output = dot(x_test, array(weights[:2], ndmin=2).T) + weights[2]
    vec_sup = sum(H_output == 1.)
    return H_output


In [9]:
def predict(h, domain=None):
    if domain == [-1., 1.]:
        for i in range(len(h)):
            if h[i][0] > 0:
                h[i][0] = 1
            else:
                h[i][0] = -1

        return h
    

In [10]:
def test(weights, x_test, y_test):
    h_output = get_foward(weights, x_test)
    y_output = predict(h_output, domain=[-1., 1.])
    accuracy = get_accuracy(y_output, array(y_test, ndmin=2).T)
    
    return accuracy

In [11]:
data = get_data("data_banknote_authentication.txt", type="csv")
number_lines = data.shape[0]
number_columns = data.shape[1]
X = array(data, ndmin=2)[:, :number_columns-1]
Y =   array(array(data, ndmin=2)[:, number_columns-1], ndmin=2).T
train_size = .8
test_size = .2

indices = where(Y == -1)
Y[indices] = 0

print(X)
print(Y)

[[  3.6216    8.6661   -2.8073   -0.44699]
 [  4.5459    8.1674   -2.4586   -1.4621 ]
 [  3.866    -2.6383    1.9242    0.10645]
 ...
 [ -3.7503  -13.4586   17.5932   -2.7771 ]
 [ -3.5637   -8.3827   12.393    -1.2823 ]
 [ -2.5419   -0.65804   2.6842    1.1952 ]]
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [12]:
for realization in range(5):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    
    weights = train(x_train, y_train)
    accuracy = test(weights, x_test, y_test)
    
    print(string.RUN.format(realization, None, accuracy, None))    

ValueError: operands could not be broadcast together with shapes (1,1097) (1097,5) 